Human genome regulatory features were collected for hg37 to match the coordinates given for EPIC array CpG as given in hg37. The regulatory build (https://europepmc.org/articles/PMC4407537 http://grch37.ensembl.org/info/genome/funcgen/regulatory_build.html) was downloaded using biomart. CpGs on the EPIC were annotated as overlapping the 6 regulatory region described or not overlapping any annotated region.

In [2]:
### EPIC annotation with Reg feature
import pandas as pd
from numpy import genfromtxt
from itertools import chain
import sys
from collections import Counter
import functools

In [3]:
import os
os.chdir('../')
os.getcwd()

'/home/redgar/Documents/organoid_passage_DNAm'

In [13]:
Feature_bed = 'data/human_regulatory_features_GRCh37p13.txt'
background = 'data/passage_background.csv'

Feature_bed = pd.read_csv(Feature_bed, header=None, names=['chr','start','end','Feature'],skiprows=1)
CpG_background_pd = pd.read_csv(background)

Feature_bed.head(3)

,chr,start,end,Feature
0,18,41514401,41515600,Promoter Flanking Region
1,12,90864334,90864573,Open chromatin
2,6,55951402,55952653,Promoter Flanking Region


In [14]:
CpG_background_pd.head(3)

,IlmnID,CHR,MAPINFO
0,cg07881041,19,5236016
1,cg18478105,20,61847650
2,cg23229610,1,6841125


Pulling a range of CpGs for an example of the annotation. The actual code takes the reange as arguments as commented out below.

In [15]:
CpG_start = 0
CpG_end = 10

#CpG_start = int(sys.argv[1])
#CpG_end = int(sys.argv[1])+10000

# subset to system arguments
CpG_background_pd = CpG_background_pd[CpG_start:CpG_end]

In [16]:
# make merge object to fill missing TFs to 0 if any
features = Feature_bed
features['count'] = 0
features = features[['Feature','count']]
features = pd.DataFrame.drop_duplicates(features)

In [17]:
features.head(3)

,Feature,count
0,Promoter Flanking Region,0
1,Open chromatin,0
3,CTCF Binding Site,0


In [18]:
### Actual background SNPs in TFBS
EPIC_features=[]

for snp, row in CpG_background_pd.iterrows():
    # match chromosome
    chr_features = Feature_bed.loc[Feature_bed['chr'] == str(row['CHR'])]

    features_snp_in = []
    for feat in range(1,len(chr_features.index)):#chr_TFBS.size
        if chr_features['start'].iloc[feat] <= row['MAPINFO'] <= chr_features['end'].iloc[feat]:
            features_snp_in.append(chr_features['Feature'].iloc[feat])

    EPIC_features.append([row['IlmnID'], features_snp_in])

In [20]:
df = pd.DataFrame(EPIC_features)
df

,0,1
0,cg07881041,[]
1,cg18478105,"[CTCF Binding Site, Promoter]"
2,cg23229610,[]
3,cg03513874,[]
4,cg05451842,"[CTCF Binding Site, Promoter]"
5,cg14797042,[Promoter Flanking Region]
6,cg09838562,"[CTCF Binding Site, Promoter]"
7,cg25458538,[Promoter Flanking Region]
8,cg14361672,"[Promoter, CTCF Binding Site]"
9,cg01763666,[]


In [ ]:
df.to_csv("data/annotation_split/EPIC_Features" + sys.argv[1] + ".csv") 